Hier können einige Konfigurationen für das Skript vorgenommen werden

In [1]:
IMAGE_SIZE = (128, 128)
IMAGE_SHAPE = (128, 128, 3)
BATCH_SIZE = 64
EPOCHS = 5

In diesem Schritt laden wir die Daten vom Dateisystem. Durch die `image_dataset_from_directory` von Keras können Daten, welche in einem bestimmten Format vorliegen, automatisch in ein brauchbares Image-Datenset umgewandelt werden. Die Bilder werden automatisch mit one-hot-encoded labels versehen und in Trainings- und Validierungsdaten aufgeteilt. Zusätzlich werden sie gepadded auf 512x512. 

In [2]:
import os
os.environ["KERAS_BACKEND"] = "torch"
import keras_core as keras

dataset = keras.utils.image_dataset_from_directory(
        os.path.join("data", "images"),
        labels="inferred",
        label_mode="int",
        color_mode="rgb",
        batch_size=BATCH_SIZE,
        image_size=IMAGE_SIZE,
        shuffle=True,
        interpolation="bilinear",
        crop_to_aspect_ratio=True,
        validation_split=0.2,
        subset="both",
        seed=42,
    )
data_train = dataset[0]
data_val = dataset[1]

data_val, data_train

Using TensorFlow backend
Found 9100 files belonging to 7 classes.
Using 7280 files for training.
Using 1820 files for validation.


(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>)

[Das RandAugment Paper](https://www.arxiv.org/abs/1909.13719) hat gezeigt, dass das zufällige verändern von Bildern bezüglicher ihrer Merkmale bei der Image-Klassifikation helfen kann. Hier fügen wir zufällig eine Reihe an `Augmentations` durch um die Bilder alle etwas anzupassen. Das sollte overfitting verhindern.

In [3]:
import keras_cv as krc

rand_augment = krc.layers.RandAugment(
    value_range=(0, 255),
    augmentations_per_image=1,
    magnitude=0.1,
    magnitude_stddev=0.1,
    rate=1,
    geometric=True
)

data_train = data_train.map(lambda x, y: (rand_augment(x), y))
data_val = data_val.map(lambda x, y: (rand_augment(x), y))

data_train, data_val

ValueError: in user code:

    File "C:\Users\nijog\AppData\Local\Temp\ipykernel_4876\10076603.py", line 12, in None  *
        lambda x, y: (rand_augment(x), keras.utils.to_categorical(y))
    File "M:\apps\anaconda\envs\test123\Lib\site-packages\keras_core\src\utils\numerical_utils.py", line 91, in to_categorical  *
        return backend.nn.one_hot(x, num_classes)
    File "M:\apps\anaconda\envs\test123\Lib\site-packages\keras_core\src\backend\tensorflow\nn.py", line 419, in one_hot  *
        return tf.one_hot(x, num_classes, axis=axis, dtype=dtype)

    ValueError: Tried to convert 'depth' to a tensor and failed. Error: None values not supported.


Hier nehmen wir die ersten 9 Bilder aus dem ersten Batch und stellen sie visuell dar. Das hilft zu verstehen bei den Augmentations genau. 

In [ ]:
from keras_cv.src.visualization import plot_image_gallery

batch = next(iter(data_train.take(1)))
images, labels = batch
plot_image_gallery(
    batch[0],
    rows=3,
    cols=3,
    value_range=(0, 255),
    show=True,
)

train cnn

In [ ]:
model = keras.Sequential([
    keras.layers.Input(shape=IMAGE_SHAPE),
    keras.layers.Conv2D(32, (4, 4), activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Conv2D(64, (4, 4), activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Conv2D(128, (4, 4), activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Conv2D(256, (4, 4), activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(7, activation='softmax'),
])
model.compile(
  optimizer=keras.optimizers.Adam(learning_rate=0.01),
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'],
)
model.summary()

In [ ]:
from livelossplot import PlotLossesKeras

history = model.fit(
    x=data_train,
    validation_data=data_val,
    epochs=EPOCHS,
    callbacks=[PlotLossesKeras()],
)